##Libraries and environment preparation

In [2]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs/

In [1]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Wed Mar 23 17:00:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "microsoft/prophetnet-large-uncased"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

##Load the dataset

In [6]:
# import dataset
raw_datasets = datasets.load_dataset("cnn_dailymail", "3.0.0")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

##Preprocess the data

In [8]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [9]:
# tokenlize inputs into map

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_train = raw_datasets["train"].select(range(143556)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["article", "highlights", "id"])

  0%|          | 0/144 [00:00<?, ?ba/s]

In [11]:
tokenized_val = raw_datasets["validation"].select(range(1000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["article", "highlights", "id"])

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 143556
})

##Fine-tuning the model

In [13]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, use_cache=False)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:357: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

In [14]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-cnn",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy = "epoch",
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    #report_to="wandb",
)

In [15]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `ProphetNetForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 143556
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 35889
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: shusunny (use `wandb logi

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.486100,2.463739,34.729500,14.687200,25.016600,31.416100,62.033000


The following columns in the evaluation set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `ProphetNetForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to prophetnet-large-uncased-finetuned-cnn/checkpoint-35889
Configuration saved in prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/config.json
Model weights saved in prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/pytorch_model.bin
tokenizer config file saved in prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/tokenizer_config.json
Special tokens file saved in prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=35889, training_loss=2.6938136667648425, metrics={'train_runtime': 35671.962, 'train_samples_per_second': 4.024, 'train_steps_per_second': 1.006, 'total_flos': 1.5829389900570624e+17, 'train_loss': 2.6938136667648425, 'epoch': 1.0})

In [18]:
wandb.finish()

eval/gen_len,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/rougeLsum,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


In [22]:
!ls -lh

total 4.1G
drwx------ 5 root root 4.0K Mar 23 17:00 drive
drwxr-xr-x 4 root root 4.0K Mar 24 03:40 prophetnet-large-uncased-finetuned-cnn
-rw-r--r-- 1 root root 4.1G Mar 24 03:51 prophetnet-large-uncased-finetuned-cnn.zip
drwxr-xr-x 1 root root 4.0K Mar  9 14:48 sample_data
drwxr-xr-x 3 root root 4.0K Mar 23 17:45 wandb


In [21]:
!zip -r prophetnet-large-uncased-finetuned-cnn.zip /content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/

  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/ (stored 0%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/special_tokens_map.json (deflated 37%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/config.json (deflated 60%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/optimizer.pt (deflated 9%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/rng_state.pth (deflated 27%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/trainer_state.json (deflated 80%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/tokenizer_config.json (deflated 38%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/pytorch_model.bin (deflated 7%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-35889/scaler.pt (deflated 55%)
  adding: content/prophetnet-large-uncased-finetuned-cnn/checkpoint-358

In [ ]:
!cp prophetnet-large-uncased-finetuned-cnn.zip '/content/drive/My Drive/weights/'